# Validación Lexicográfica

En esta notebook, analicemos con los datos etiquetados de la academia cómo quedó cada métrica.

Para ello, integraremos los datos del último etiquetado de la Academia, que es el localizado en `data/listado_definitivo_2.csv`


In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import contrastes

df = contrastes.read_occurrence_dataframe("output/listados/listado_completo.csv")
nuevas_etiquetadas = pd.read_csv("data/listado_definitivo_2.csv")

nuevas_etiquetadas.set_index("palabra", inplace=True)


Populating the interactive namespace from numpy and matplotlib


/home/jmperez/projects/contrastes/notebooks/contrastes/processing.py:147: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.cant_palabras = cant_palabras
/home/jmperez/projects/contrastes/notebooks/contrastes/processing.py:148: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.cant_personas = cant_personas


Las que sean 0.5, las pongo como uno...

In [2]:
df.loc[df.etiqueta == 0.5, "etiqueta"] = 1.0

In [3]:
print("Cantidad total de palabras: {}".format(df.shape[0]))
print("Cantidad de palabras etiquetadas: {}".format(df.etiqueta.notna().sum()))

Cantidad total de palabras: 136847
Cantidad de palabras etiquetadas: 4853


Ahora, veamos sólo las 1000 primeras de cada métrica, que son las que nos importan

In [4]:
listado_total = df[(df.rank_palabras <= 1000) | (df.rank_personas <= 1000) | (df.rank_palper <= 1000)].copy()

rank_cols = [c for c in listado_total.columns if "rank_" in c]

for col in rank_cols:
    listado_total[col] = listado_total[col].astype("int")
    
listado_total.shape

(1380, 108)

Ok, 1380. ¿Cuántas quedan sin etiquetar?

In [5]:
faltantes = listado_total[listado_total.etiqueta.isna()]

print("Faltan etiquetar {} palabras".format(faltantes.shape[0]))
print("De estas, {} son lugares".format(sum(faltantes.es_lugar)))

Faltan etiquetar 460 palabras
De estas, 22 son lugares


¿Y cuantas tenemos en el nuevo etiquetado?

In [6]:
nuevas_etiquetadas.shape

(460, 4)

¿Están todas?

In [7]:
faltantes.index.difference(nuevas_etiquetadas.index)

Index([], dtype='object', name='palabra')

Así parece, están todas, iupi!

In [9]:
df.loc[faltantes.index, "etiqueta"] = nuevas_etiquetadas["candidata"].astype("float64")
listado_total.loc[faltantes.index, "etiqueta"] = nuevas_etiquetadas["candidata"].astype("float64")

listado_total["etiqueta"] = listado_total["etiqueta"].astype("int")

In [10]:
listado_palabras = listado_total[listado_total.rank_palabras <= 1000]
listado_personas = listado_total[listado_total.rank_personas <= 1000]
listado_palper = listado_total[listado_total.rank_palper <= 1000]
listado_palabras.shape, listado_personas.shape, listado_palper.shape

((1000, 108), (1000, 108), (1000, 108))

In [11]:
p_palabras = listado_palabras.etiqueta.mean()
p_personas = listado_personas.etiqueta.mean()
p_palper = listado_palper.etiqueta.mean()

print("Porcentaje palabras          = {:.2f}%".format(p_palabras*100))
print("Porcentaje personas          = {:.2f}%".format(p_personas*100))
print("Porcentaje palabras-personas = {:.2f}%".format(p_palper*100))


Porcentaje palabras          = 21.90%
Porcentaje personas          = 30.20%
Porcentaje palabras-personas = 25.30%


La mejor métrica para esto quedó en contar personas

In [12]:
etiquetadas = df[df.etiqueta.notna()]

(etiquetadas.rank_personas <= 5000).sum()

2548

**Pregunta**: Me genera confusión es esto: ¿por qué quedaron sólo 2.5K etiquetadas de las 5000 primeras? :-\

**Respuesta**: Es que simplificamos bastante la métrica!

Me quedo sólo con el listado, sus rankings y etiquetas. Primero, guardemos esto para compartir con la gente de la AAL

In [13]:
columnas = ["rank_palabras", "rank_personas", "rank_palper", "etiqueta"] + df.cant_palabras
listado_total.to_csv("output/listados/1000_etiquetadas.csv", columns=columnas)

In [14]:
pd.set_option('display.max_rows', 500)

listado_etiquetado = listado_total[
    ["rank_palabras", "rank_personas", "rank_palper", "etiqueta"]].copy()

regionalismos = listado_etiquetado[listado_etiquetado.etiqueta == 1]

regionalismos

,rank_palabras,rank_personas,rank_palper,etiqueta
palabra,,,,
agraden,107,75042,9848,1
aij,175,59,76,1
aije,901,385,531,1
aijue,55,5,11,1
aijuee,1394,468,788,1
aijueee,692,130,280,1
aiss,1949,835,1282,1
akc,686,1895,1123,1
alakalufes,861,1180,916,1


## Overlaps

Calculemos el overlap entre cada ranking

In [16]:
def overlap(r1, r2):
    return df[(df[r1] <= 1000) & (df[r2] <= 1000) ].shape[0] / 10

comparaciones = [
    ("rank_palabras", "rank_personas"), 
    ("rank_personas", "rank_palper"),
    ("rank_palabras", "rank_palper")
]

for r1, r2 in comparaciones:
    print("{} - {} overlap ===> {}".format(r1, r2, overlap(r1, r2)))

rank_palabras - rank_personas overlap ===> 63.1
rank_personas - rank_palper overlap ===> 83.0
rank_palabras - rank_palper overlap ===> 79.0
